In [1]:
#7/5/24 For the creation of an Autoencoder for Feature Extraction
#The goal is to create an autoencoder that can parse SMILES via one hot encoding or a graphical representation
#Main steps:
    #One hot encoding SMILES DONE
    #import into autoencoder
    #train to recreate SMILES
    #use latent space for feature prediction
#Mostly for testing, one hot data is too large to parse into jupyter
#Will test on drug target data
#Even though an autoencoder model was already developed, due to the poor results it has shown
#It has been totally rewritten

import pandas as pd
import numpy as np
#Data import
targetdata = pd.read_csv('/Users/james/Documents/Honours/Python/Target_Selection/Drug_Targets/DrugTargets.csv')


####################
#Todo: iterate over multiple epoch size, batchsize and latent space sizes
#maintain seed
#store best model








In [2]:
#Creating test/train splits
from sklearn.model_selection import train_test_split

trainset, testset = train_test_split(targetdata, test_size=0.2, random_state=81)
#Converts Toxicity Values to a list
ytrain = trainset['Toxicity_Values'].to_list
ytest = testset['Toxicity_Values'].to_list
#Converts encoded drug target values to an array
xtrain = trainset.iloc[:, 2:].values
xtest = testset.iloc[:, 2:].values

In [7]:
#Determines layer size and input/output
#Here it turns the 334 one hot encoded features to 42
import tensorflow as tf
from tensorflow import keras
latent_space = 42
keras.utils.set_random_seed(81)

#Encoder Creation
enc_in = keras.Input(shape=(334, 1))
x = keras.layers.Flatten()(enc_in)
enc_out = keras.layers.Dense(latent_space, activation="relu")(x)
encoder = keras.Model(enc_in, enc_out, name='encoder')

#Decoder Creation
decoder_in = keras.layers.Dense(latent_space, activation='relu')(enc_out)
x = keras.layers.Dense(334, activation='relu')(decoder_in)
decoder_out = keras.layers.Reshape((334,1))(x)

#Optimiser
optimise = tf.keras.optimizers.legacy.Adam()

#Model Creation
autoencoder = keras.Model(enc_in, decoder_out, name='autoencoder')

autoencoder.compile(optimise, loss='binary_crossentropy')
autoencoder.summary()

Model: "autoencoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 334, 1)]          0         
                                                                 
 flatten_1 (Flatten)         (None, 334)               0         
                                                                 
 dense_3 (Dense)             (None, 42)                14070     
                                                                 
 dense_4 (Dense)             (None, 42)                1806      
                                                                 
 dense_5 (Dense)             (None, 334)               14362     
                                                                 
 reshape_1 (Reshape)         (None, 334, 1)            0         
                                                                 
Total params: 30,238
Trainable params: 30,238
Non-train

In [10]:
#Runs and creates the autoencoder
epochs = 7
batchsize = 3
for epoch in range(epochs):
    
    logs = autoencoder.fit(
    xtrain,
    xtrain,
    epochs=1,
    batch_size=batchsize, validation_split=0.2)
    
    autoencoder.save(str(f"models/AE-{epoch+1}.model"))



2826/2826 [==============================] - 2s 639us/step - loss: 0.1506 - val_loss: 0.1401


2024-05-14 22:14:52.032314: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,334]
	 [[{{node inputs}}]]
2024-05-14 22:14:52.111880: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,334]
	 [[{{node inputs}}]]


INFO:tensorflow:Assets written to: models/AE-1.model/assets
2826/2826 [==============================] - 2s 632us/step - loss: 0.1531 - val_loss: 0.1482


2024-05-14 22:14:54.165738: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,334]
	 [[{{node inputs}}]]
2024-05-14 22:14:54.244926: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,334]
	 [[{{node inputs}}]]


INFO:tensorflow:Assets written to: models/AE-2.model/assets
2826/2826 [==============================] - 2s 639us/step - loss: 0.1472 - val_loss: 0.1410


...

INFO:tensorflow:Assets written to: models/AE-5.model/assets
2826/2826 [==============================] - 2s 646us/step - loss: 0.1624 - val_loss: 0.1704


2024-05-14 22:15:02.875950: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,334]
	 [[{{node inputs}}]]
2024-05-14 22:15:02.954736: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,334]
	 [[{{node inputs}}]]


INFO:tensorflow:Assets written to: models/AE-6.model/assets
2826/2826 [==============================] - 2s 632us/step - loss: 0.1630 - val_loss: 0.1756


2024-05-14 22:15:05.006412: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,334]
	 [[{{node inputs}}]]
2024-05-14 22:15:05.085261: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,334]
	 [[{{node inputs}}]]


INFO:tensorflow:Assets written to: models/AE-7.model/assets


In [20]:
#Gathers the accuracy of the autoencoder on a test set
ae_out = autoencoder.predict([ xtest.reshape(-1, 334, 1) ])
test = ae_out.reshape(2649,334)

ids = 0
acc = []
fplist = []
tplist = []

for things in test:
    guesses = []
    testset = []
    
    loops = 0
    for items in xtest[ids]:
        if items != 0:
            testset.append(loops)
        loops = loops + 1
    
    loops = 0
    for items in test[ids]:
        if items > 0.5:
            guesses.append(loops)
        loops = loops + 1
        
    correct = 0
    loops = 0
    tp = 0
    fp = 0
    for items in guesses:
        if items in testset:
            correct = correct + 1
            tp = tp + 1
        else:
            fp = fp + 1
        loops = loops + 1
    
    accuracy = correct / loops
    tprate = tp / len(guesses)
    fprate = fp / len(guesses)
    
    acc.append(accuracy)
    tplist.append(tprate)
    fplist.append(fprate)
    
    ids = ids + 1

totalacc = 0
totaltp = 0
totalfp = 0

loops = 0
for items in acc:
    totalacc = totalacc + items
    loops = loops + 1
netacc = totalacc/loops

loops = 0
for items in tplist:
    totaltp = totaltp + items
    loops = loops + 1
nettp = totaltp/loops

loops = 0
for items in fplist:
    totalfp = totalfp + items
    loops = loops + 1
netfp = totalfp/loops
    
netacc = totalacc/loops
nettp = totaltp/loops
netfp = totalfp/loops

print('net accuracy =', netacc)
print('fp rate =', netfp)
print('tp rate =', nettp)

83/83 [==============================] - 0s 490us/step
net accuracy = 0.8242661358036444
fp rate = 0.17573386419635623
tp rate = 0.8242661358036444


In [35]:
#For spot testing an individual sample
loops = 0
count = 0
ids = 11
testset = []
guesses = []

for items in test[ids]:
    if items > 0.5:
        count = count + 1
        guesses.append(loops)
        #print(loops, items)
    loops = loops + 1

#print('count =', count)

#print('----------------')

loops = 0
count = 0
for items in xtest[ids]:
    if items != 0:
        count = count + 1
        testset.append(loops)
        #print(loops)
    loops = loops + 1

acc = 0
for items in guesses:
    if items in testset:
        acc = acc + 1

accuracy = acc / len(guesses)
#print('count =', count)
print('accuracy =', accuracy)

accuracy = 1.0
